In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd 
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle
from csv import DictReader

# from utils.dataset import DataSet
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM, Dense
from keras.models import Sequential
import nltk

nltk.download('punkt')
nltk.download('wordnet')



from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from numpy import zeros

from keras import callbacks
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
# import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf

from numpy import asarray
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import punctuation

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Specify the folder locations
W2V_DIR = './data/GoogleNews-vectors-negative300.bin' # W2v
GloVe_DIR = './data/glove.6B.300d.txt' #Glove
FastTxt_DIR = './data/wiki-news-300d-1M.vec' #fasttext



In [ ]:

#the data directory
MAX_SENT_LEN = 150      #75(0.68), 150, 300 700(90% but too time comsuming)
MAX_VOCAB_SIZE = 40000 #vocabulary
LSTM_DIM = 128#len(embd[0])
EMBEDDING_DIM = 300 #50 for GloVe 300 for w2v
BATCH_SIZE = 128
N_EPOCHS = 40 

In [ ]:
class DataSet():
    def __init__(self, name="train", path="./data"):
        self.path = path

        print("Reading dataset")
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        self.stances = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows


In [ ]:
def generate_features(stances,dataset):
    h, b, y = [],[],[]
    LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
    for stance in stances:

        # print(stance['Stance'])
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])
   

    return(h,b,y)


In [ ]:
def generate_features_new(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    return y
  

In [ ]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    print("Score",score)
    best_score, _ = score_submission(actual,actual)
    # print("best_Score",best_score)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score


In [ ]:
d=DataSet()
h,b,y= generate_features(d.stances,d)

combined_list = [list(x) for x in zip(h, b,y)]

combine_df_train=pd.DataFrame(combined_list,columns = ["Headline","articleBody","Stances"])
combine_df_train.to_csv("./data/combined_list_1.csv")



Reading dataset
Total stances: 49972
Total bodies: 1683


In [ ]:
competition_dataset = DataSet("competition_test")
h_test,b_test,y_test= generate_features(competition_dataset.stances,competition_dataset)

combined_list_test = [list(x_test) for x_test in zip(h_test, b_test, y_test)]

combine_df_test=pd.DataFrame(combined_list_test,columns = ["Headline","articleBody","Stances"])
combine_df_test.to_csv("./data/combined_list_test.csv")



Reading dataset
Total stances: 25413
Total bodies: 904


In [ ]:
# Pre-processing involves removal of puctuations and converting text to lower case
word_seq_head_train = [text_to_word_sequence(head) for head in combine_df_train['Headline']]
word_seq_bodies_train = [text_to_word_sequence(body) for body in combine_df_train['articleBody']]
word_seq_head_test = [text_to_word_sequence(head) for head in combine_df_test['Headline']]
word_seq_bodies_test = [text_to_word_sequence(body) for body in combine_df_test['articleBody']]

In [ ]:
word_seq = []
for i in range(len(word_seq_head_train)):
    word_seq.append(word_seq_head_train[i])
for i in range(len(word_seq_bodies_train)):
    word_seq.append(word_seq_bodies_train[i])


# word_seq_test_1 = []
for i in range(len(word_seq_head_test)):
    word_seq.append(word_seq_head_test[i])
for i in range(len(word_seq_bodies_test)):
    word_seq.append(word_seq_bodies_test[i])

In [ ]:
filter_list = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters=filter_list)
tokenizer.fit_on_texts([seq for seq in word_seq])
#because it only includes unique words(tokens)

print("Number of words in vocabulary:", len(tokenizer.word_index))

Number of words in vocabulary: 35978


In [ ]:
# Combine headline and body together
word_seq_train = [list(i) for i in word_seq_head_train]
for i in range(len(word_seq_head_train)):
    word_seq_train[i].extend(word_seq_bodies_train[i]) 
    
word_seq_test = [list(i) for i in word_seq_head_test]
for i in range(len(word_seq_head_test)):
    word_seq_test[i].extend(word_seq_bodies_test[i])

In [ ]:
# Shorten the sentence to a fixed length
# Convert the sequence of words to sequnce of indices
X_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_train])
X_train = pad_sequences(X_train, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
y_train_1 = combine_df_train['Stances']


X_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_test])
X_test = pad_sequences(X_test, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
y_test_1 = combine_df_test['Stances']


# Encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(y_train_1)
encoded_train = encoder_train.transform(y_train_1)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_train)

In [ ]:
# Encode class values as integers
encoder_test = LabelEncoder()
encoder_test.fit(y_test_1)
encoded_test = encoder_test.transform(y_test_1)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_test)

GLOVE

In [ ]:
embeddings_index = dict()
f = open(GloVe_DIR)
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((len(tokenizer.word_index)+1, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


W2v

In [ ]:
def embed_matrix(model,word_index,EMBEDDING_DIM):
  embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0
  for word, i in word_index.items(): # i=0 is the embedding for the zero padding
      try:
          embeddings_vector = model.wv.get_vector(word)
      except KeyError:
          embeddings_vector = None
          #none: if words in sentence don't have pre-trained corresponding embedding, then error occurs

      if embeddings_vector is not None:
          embeddings_matrix[i] = embeddings_vector
          #if pre-trained word embedding exists，then let embeddings_matrix[i] is this embedding
          #Wi:the ith row of embeddings_matrix

  return embeddings_matrix

In [ ]:
# Load the word2vec embeddings 

embeddings_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_DIR, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
embeddings_matrix_w2v= embed_matrix(embeddings_w2v,tokenizer.word_index,EMBEDDING_DIM)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


FastText

In [ ]:
# Load the word2vec embeddings 
embeddings_fasttext = gensim.models.KeyedVectors.load_word2vec_format(FastTxt_DIR)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
embeddings_matrix_fast_text= embed_matrix(embeddings_fasttext,tokenizer.word_index,EMBEDDING_DIM)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [ ]:
embeddings_matrix_fast_text.shape

(35979, 300)

BiDirectional Model

In [ ]:
def model(word_index,EMBEDDING_DIM,embeddings_matrix):
  # Build a sequential model by stacking neural net units 
  model_1 = Sequential()
  model_1.add(Embedding(input_dim=len(word_index)+1,
                            output_dim=EMBEDDING_DIM,
                            weights = [embeddings_matrix], trainable=True, name='word_embedding_layer', mask_zero=True))
  #mask_zero is to deal with padding problem
  #model.add(Bidirectional(LSTM(LSTM_DIM, return_sequences=False, name='lstm_layer1'))) #bi(lstm)
  model_1.add(Bidirectional(LSTM(LSTM_DIM, return_sequences=False, name='Bidrectional_lstm_layer1')))
  
  model_1.add(Dropout(rate=0.8, name='dropout_1')) # Can try varying dropout rates, in paper suggest 0.8
  model_1.add(Dense(4, activation='softmax', name='output_layer'))
  model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model_1

W2v

In [ ]:
w2v_model=model(tokenizer.word_index,EMBEDDING_DIM,embeddings_matrix_w2v)

In [ ]:
import pickle

In [ ]:
filepath="/content/drive/My Drive/MSCI_641/best_model_w2v.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, 
                                       monitor='val_accuracy', 
                                       verbose=1,mode='max', 
                                       save_best_only=True)

callbacks_list1 = [checkpoint]

history_w2v = w2v_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=5, validation_data=(X_test, y_test),callbacks = callbacks_list1)


with open('/content/drive/My Drive/MSCI_641/history_w2v.pkl', 'wb') as file_pi:
  pickle.dump(history_w2v.history, file_pi)

Epoch 1/5
391/391 [==============================] - ETA: 0s - loss: 0.6625 - accuracy: 0.7667
Epoch 00001: val_accuracy improved from -inf to 0.70094, saving model to /content/drive/My Drive/MSCI_641/best_model_w2v.hdf5
391/391 [==============================] - 119s 303ms/step - loss: 0.6625 - accuracy: 0.7667 - val_loss: 0.8188 - val_accuracy: 0.7009
Epoch 2/5
391/391 [==============================] - ETA: 0s - loss: 0.4417 - accuracy: 0.8351
Epoch 00002: val_accuracy did not improve from 0.70094
391/391 [==============================] - 105s 268ms/step - loss: 0.4417 - accuracy: 0.8351 - val_loss: 0.9094 - val_accuracy: 0.6332
Epoch 3/5
391/391 [==============================] - ETA: 0s - loss: 0.3197 - accuracy: 0.8770
Epoch 00003: val_accuracy did not improve from 0.70094
391/391 [==============================] - 107s 274ms/step - loss: 0.3197 - accuracy: 0.8770 - val_loss: 0.8412 - val_accuracy: 0.6820
Epoch 4/5
391/391 [==============================] - ETA: 0s - loss: 0.241

In [ ]:
X_train.shape

(49972, 150)

In [ ]:
X_test.shape

(25413, 150)

In [ ]:
y_test.shape

(25413, 4)

In [ ]:
predicted = w2v_model.predict(X_test)
predicted

array([[5.73227473e-04, 1.96006094e-05, 4.52221837e-03, 9.94884908e-01],
       [1.16999354e-03, 1.34824397e-04, 1.73781591e-03, 9.96957421e-01],
       [1.10327015e-02, 1.13212714e-04, 3.59811052e-03, 9.85255957e-01],
       ...,
       [8.77932012e-02, 1.52423289e-02, 1.96057290e-01, 7.00907111e-01],
       [9.43385251e-03, 8.21836176e-04, 2.90959328e-01, 6.98784947e-01],
       [4.29116422e-03, 1.80053030e-04, 2.66913953e-03, 9.92859721e-01]],
      dtype=float32)

In [ ]:
y_true = np.argmax(y_test,axis = 1)

In [ ]:
pred_new = np.argmax(predicted,axis = 1)

In [ ]:
pred

array([3, 3, 3, ..., 3, 3, 3])

In [ ]:


competition_dataset = DataSet("competition_test")
y_competition = generate_features_new(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

# Load/Precompute all features now
#Run on competition dataset
actual = [LABELS[int(a)] for a in y_competition]
predict_1= [LABELS[int(a)] for a in pred_new]
print(actual)
print(predict_1)

print("Scores on the test set")
report_score(actual,predict_1)

Reading dataset
Total stances: 25413
Total bodies: 904
['unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'agree', 'unrelated', 'discuss', 'unrelated', 'discuss', 'agree', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'discuss', 'agree', 'agree', 'disagree', 'discuss', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'agree', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'agree', 'unrelated', 'unrelated', 'unrelated', 'agree', 'unrelated', 'agree', 'unrelated', 'agree', 'unrelated', 'discuss', 'unr

51.67042162858063

GLOVE

In [ ]:
glove_model=model(tokenizer.word_index,EMBEDDING_DIM,embedding_matrix)

In [ ]:
filepath="/content/drive/My Drive/MSCI_641/best_model_glove.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, 
                                       monitor='val_accuracy', 
                                       verbose=1,mode='max', 
                                       save_best_only=True)

callbacks_list1 = [checkpoint]


#%%capture history
#using different emedding pretrained dataset
history_glove = glove_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=5, validation_data=(X_test, y_test),callbacks = callbacks_list1)
with open('/content/drive/My Drive/MSCI_641/history_glove.pkl', 'wb') as file_pi:
  pickle.dump(history_glove.history, file_pi)

Epoch 1/5
391/391 [==============================] - ETA: 0s - loss: 0.6934 - accuracy: 0.7543
Epoch 00001: val_accuracy improved from -inf to 0.72526, saving model to /content/drive/My Drive/MSCI_641/best_model_glove.hdf5
391/391 [==============================] - 112s 287ms/step - loss: 0.6934 - accuracy: 0.7543 - val_loss: 0.7830 - val_accuracy: 0.7253
Epoch 2/5
391/391 [==============================] - ETA: 0s - loss: 0.4311 - accuracy: 0.8388
Epoch 00002: val_accuracy did not improve from 0.72526
391/391 [==============================] - 105s 268ms/step - loss: 0.4311 - accuracy: 0.8388 - val_loss: 0.8848 - val_accuracy: 0.6400
Epoch 3/5
391/391 [==============================] - ETA: 0s - loss: 0.2950 - accuracy: 0.8880
Epoch 00003: val_accuracy did not improve from 0.72526
391/391 [==============================] - 106s 272ms/step - loss: 0.2950 - accuracy: 0.8880 - val_loss: 0.8618 - val_accuracy: 0.6900
Epoch 4/5
391/391 [==============================] - ETA: 0s - loss: 0.2

In [ ]:
predicted = glove_model.predict(X_test)
predicted

array([[4.7672535e-03, 3.4690159e-04, 5.4926896e-03, 9.8939312e-01],
       [2.0437049e-04, 2.0118618e-05, 8.3691455e-05, 9.9969184e-01],
       [1.2972778e-04, 1.9475783e-06, 1.3715713e-04, 9.9973112e-01],
       ...,
       [7.7637863e-01, 1.2047652e-01, 7.5637117e-02, 2.7507709e-02],
       [1.3936818e-01, 1.3141203e-02, 2.5066448e-02, 8.2242417e-01],
       [3.0976972e-01, 1.8991478e-02, 2.1019105e-02, 6.5021968e-01]],
      dtype=float32)

In [ ]:
pred_new_glove = np.argmax(predicted,axis = 1)

In [ ]:
competition_dataset = DataSet("competition_test")
y_competition = generate_features_new(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

actual = [LABELS[int(a)] for a in y_competition]
predict_1= [LABELS[int(a)] for a in pred_new_glove]

print("Scores on the test set")
report_score(actual,predict_1)

Reading dataset
Total stances: 25413
Total bodies: 904
Scores on the test set
Score 6480.75
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    404    |     2     |    372    |   1125    |
-------------------------------------------------------------
| disagree  |    69     |     1     |    126    |    501    |
-------------------------------------------------------------
|  discuss  |    251    |     0     |   2052    |   2161    |
-------------------------------------------------------------
| unrelated |    877    |     2     |   2195    |   15275   |
-------------------------------------------------------------
Score: 6480.75 out of 11651.25	(55.62278725458642%)


55.62278725458642

FastText

In [ ]:
fasttext_model=model(tokenizer.word_index,EMBEDDING_DIM,embeddings_matrix_w2v)

In [ ]:
filepath="/content/drive/My Drive/MSCI_641/best_model_fasttext.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, 
                                       monitor='val_accuracy', 
                                       verbose=1,mode='max', 
                                       save_best_only=True)

callbacks_list1 = [checkpoint]

history_fast = fasttext_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=5, validation_data=(X_test, y_test),callbacks = callbacks_list1)

with open('/content/drive/My Drive/MSCI_641/history_fasttext.pkl', 'wb') as file_pi:
  pickle.dump(history_fast.history, file_pi)

Epoch 1/5
391/391 [==============================] - ETA: 0s - loss: 0.6581 - accuracy: 0.7680
Epoch 00001: val_accuracy improved from -inf to 0.69295, saving model to /content/drive/My Drive/MSCI_641/best_model_fasttext.hdf5
391/391 [==============================] - 112s 287ms/step - loss: 0.6581 - accuracy: 0.7680 - val_loss: 0.8276 - val_accuracy: 0.6930
Epoch 2/5
391/391 [==============================] - ETA: 0s - loss: 0.4432 - accuracy: 0.8359
Epoch 00002: val_accuracy did not improve from 0.69295
391/391 [==============================] - 105s 268ms/step - loss: 0.4432 - accuracy: 0.8359 - val_loss: 0.8161 - val_accuracy: 0.6917
Epoch 3/5
391/391 [==============================] - ETA: 0s - loss: 0.3306 - accuracy: 0.8727
Epoch 00003: val_accuracy did not improve from 0.69295
391/391 [==============================] - 105s 268ms/step - loss: 0.3306 - accuracy: 0.8727 - val_loss: 0.8485 - val_accuracy: 0.6885
Epoch 4/5
391/391 [==============================] - ETA: 0s - loss: 

In [ ]:
predicted = fasttext_model.predict(X_test)
predicted

array([[5.5687614e-02, 2.0363489e-03, 1.8930693e-01, 7.5296909e-01],
       [9.1961393e-04, 1.3105085e-04, 2.9020340e-03, 9.9604720e-01],
       [3.3178482e-02, 3.3323138e-04, 4.1192817e-03, 9.6236902e-01],
       ...,
       [4.4193244e-01, 2.4725330e-01, 2.4075252e-01, 7.0061721e-02],
       [2.1847172e-03, 5.6485337e-04, 3.1422889e-01, 6.8302155e-01],
       [6.0846776e-01, 2.5881737e-02, 3.4222439e-02, 3.3142802e-01]],
      dtype=float32)

In [ ]:
pred_new_ft = np.argmax(predicted,axis = 1)

In [ ]:
predict_1= [LABELS[int(a)] for a in pred_new_ft]

print("Scores on the test set")
report_score(actual,predict_1)

Scores on the test set
Score 5995.25
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    436    |     0     |    237    |   1230    |
-------------------------------------------------------------
| disagree  |    108    |     1     |    70     |    518    |
-------------------------------------------------------------
|  discuss  |    360    |     0     |   1644    |   2460    |
-------------------------------------------------------------
| unrelated |   1368    |     3     |   2096    |   14882   |
-------------------------------------------------------------
Score: 5995.25 out of 11651.25	(51.45585237635447%)


51.45585237635447